# Math Question Answer Verification Competition

## TEAM LLM training notebook

 ## INSTALLATION


In [ ]:
# Installations without capture
# to see the output
!pip install uv

!uv pip install  \
  "unsloth @ git+https://github.com/unslothai/unsloth.git" \
  "xformers" "trl" "peft" \
  "accelerate" "bitsandbytes" "transformers"
!uv pip install unsloth_zoo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 107.6 MB/s eta 0:00:00
Using Python 3.12.12 environment at: /usr
Resolved 65 packages in 3.67s
Prepared 4 packages in 1.89s
Installed 4 packages in 6ms
 + bitsandbytes==0.48.2
 + trl==0.24.0
 + unsloth==2025.10.12 (from git+https://github.com/unslothai/unsloth.git@d707bd43b4e883b521761d525be2fae428fe5980)
 + xformers==0.0.32.post2
Using Python 3.12.12 environment at: /usr
Resolved 84 packages in 250ms
Prepared 9 packages in 761ms
Uninstalled 4 packages in 352ms
Installed 9 packages in 17ms
 + cut-cross-entropy==25.1.1
 - datasets==4.0.0
 + datasets==4.3.0
 + msgspec==0.19.0
 - pyarrow==18.1.0
 + pyarrow==22.0.0
 + shtab==1.7.2
 - torchao==0.10.0
 + torchao==0.14.1
 - trl==0.24.0
 + trl==0.23.0
 + tyro==0.9.35
 + unsloth-zoo==2025.10.13


In [ ]:
from unsloth import FastLanguageModel
import torch
# We tried:
# 1024 - did well, some samples were of higher sequence length
# 2048 - excessive, produced 62% accuracy
# 4094 - too high, no samples have such a high seq length
# 8192 - didn't need to try it at all, RAM crash!
max_seq_length = 1048 # to fit all samples :)
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.10.12: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

## Training Prompts

In [ ]:
# We changed the training prompt to focus more on the solution
# The model did well even with the prompt in the starter notebook
# but this is even better
training_prompt = """You are a meticulous math expert. Your goal is to verify the *entire reasoning* of the solution, not just the final answer.
Read the problem, then carefully analyze each step of the solution for logical errors, calculation mistakes, or incorrect reasoning.
Based on your step-by-step analysis, determine if the solution is correct.

Answer with ONLY True or False.

### Problem:
{}

### Solution:
{}

### Output:
{}
"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    question = examples["question"]
    solution=examples["solution"]
    output= examples["is_correct"]
    texts = []
    for instruction, input, output in zip(question,solution, output):
        text = training_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

## Competition dataset

In [ ]:
from datasets import load_dataset
dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp")
dataset

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

data/train-00001-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/3.65M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'is_correct', 'answer', 'solution'],
        num_rows: 1000000
    })
    test: Dataset({
        features: ['question', 'is_correct', 'answer', 'solution'],
        num_rows: 10000
    })
})

In [ ]:
# Access the training split
train_dataset = dataset['train']

# Step 2: Shuffle the training dataset
# Setting a seed for reproducibility
shuffled_train = train_dataset.shuffle(seed=42)

# Step 3: Select the first 2,000 samples for validation
validation_set = shuffled_train.select(range(2000))

# Select the next 80,000 samples for the new training set
training_set = shuffled_train.select(range(2000, 82000))

In [ ]:
training_set = training_set.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/80000 [00:00<?, ? examples/s]

In [ ]:
#print a smaple training example
training_set['text'][0]

"You are a meticulous math expert. Your goal is to verify the *entire reasoning* of the solution, not just the final answer.\nRead the problem, then carefully analyze each step of the solution for logical errors, calculation mistakes, or incorrect reasoning.\nBased on your step-by-step analysis, determine if the solution is correct.\n\nAnswer with ONLY True or False.\n\n### Problem:\nIf Natasha was going 10 mph over the speed limit and it took her an hour to arrive at her destination that was 60 miles away, what was the speed limit?\n\n### Solution:\nLet's solve this problem using Python code.\n<llm-code>\nmph_over_limit = 10\ntime_traveled = 1\ndistance_traveled = 60\nspeed_limit = (distance_traveled + mph_over_limit * time_traveled) / time_traveled\nspeed_limit\n</llm-code>\n<llm-code-output>\n70.0\n</llm-code-output>\nThus the speed limit was \\boxed{70} mph.\n\n### Output:\nFalse\n<|end_of_text|>"

## Load model and wrap with LoRA adapters

In [ ]:
# We tried many different combinations of r, from 1 to 128
# and tried keeping lora alpha 2*r, r and r/2
# r = 32, and lora_alpha = r/2 did great!
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 500,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.10.12 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## SFT

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = training_set,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/80000 [00:00<?, ? examples/s]

## TRAINING


In [ ]:
len(training_set['text'])

80000

In [ ]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 80,000 | Num Epochs = 1 | Total steps = 10,000
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 83,886,080 of 8,114,147,328 (1.03% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.614900
2,1.704700
3,1.743100
4,1.439300
5,1.555900
6,1.631900
7,1.350300
8,1.202800
9,1.026800
10,0.928400


## SAVING MODEL

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

# Define the path to save the model checkpoint in Google Drive
save_path = "/content/drive/MyDrive/soham-model-what-a-king-plswork"

# Create the directory if it doesn't exist
os.makedirs(save_path, exist_ok=True)

# Save the model and tokenizer
trainer.model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Model checkpoint and tokenizer saved to: {save_path}")

Model checkpoint and tokenizer saved to: /content/drive/MyDrive/soham-model-what-a-king-plswork


In [ ]:
import shutil
import os
from IPython.display import FileLink

save_path = "/content/drive/MyDrive/soham-model-what-a-king-updated-2"

# Create the directory if it doesn't exist
os.makedirs(save_path, exist_ok=True)

# Save the model and tokenizer locally in the Kaggle environment
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

# Compress the saved model directory into a ZIP file
shutil.make_archive("soham-model-updated", 'zip', "soham-model-updated")

# Generate a download link for the ZIP file
# display(FileLink(r'lora_model_50.zip'))


'/content/soham-model-updated.zip'

## INFERENCE

## INFERENCE PROMPT

In [ ]:
inference_prompt = """You are a meticulous math expert. Your goal is to verify the *entire reasoning* of the solution, not just the final answer.
Read the problem, then carefully analyze each step of the solution for logical errors, calculation mistakes, or incorrect reasoning.
Based on your step-by-step analysis, determine if the solution is correct.

Answer with ONLY True or False.

### Problem:
{}

### Solution:
{}

### Output:
"""

def formatting_prompts_func_inference(examples):
    question = examples["question"]
    solution=examples["solution"]
    output= examples["is_correct"]
    texts = []
    for instruction, input, output in zip(question,solution, output):
        text = inference_prompt.format(instruction, input)
        texts.append(text)
    return { "text" : texts, }

## VALIDATION DATASET



In [ ]:
validation_dataset = validation_set.map(formatting_prompts_func_inference, batched = True,)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
#print a smaple training example
validation_dataset['text'][0]

"You are a great mathematician and you are tasked with finding if an solution to a given maths question is correct or not. Your response should be 'True' if solution is correct, otherwise 'False'. Below is Question and Solution.\nQuestion:\nA line is parameterized by\n\\[\\begin{pmatrix} x \\\\ y \\end{pmatrix} = \\begin{pmatrix} 2 \\\\ 3 \\end{pmatrix} + t \\begin{pmatrix} -1 \\\\ 5 \\end{pmatrix}.\\]A second line is parameterized by\n\\[\\begin{pmatrix} x \\\\ y \\end{pmatrix} = \\begin{pmatrix} 0 \\\\ 7 \\end{pmatrix} + u \\begin{pmatrix} -1 \\\\ 4 \\end{pmatrix}.\\]Find the point where the lines intersect.\nSolution:\nFirst, we need to solve the system of equations\n\\[\n\\begin{aligned}\n2 - t &= s\\\\\n3 + 5t &= 7 + 4s\n\\end{aligned}\n\\]\nby eliminating s.\nWe'll use sympy.\n<llm-code>\nfrom sympy import symbols, solve\n\n# define the variables\nt, s = symbols('t s')\n\n# define the equations\nequation1 = 2 - t - s\nequation2 = 3 + 5*t - (7 + 4*s)\n\n# solve the equations\nsolu

In [ ]:
# Running inference on single validation sample

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

example_no=0

input_prompt=validation_dataset['text'][example_no]

print("Input Promt:\n", input_prompt)
inputs = tokenizer(
[
    input_prompt
], return_tensors = "pt").to("cuda")

input_shape = inputs['input_ids'].shape
input_token_len = input_shape[1] # 1 because of batch
outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
# you can get the whole generated text by uncommenting the below line
# text_generated = tokenizer.batch_decode([outputs, skip_special_tokens=True)

response = tokenizer.batch_decode([outputs[0][input_token_len:]], skip_special_tokens=True)
response

Input Promt:
 You are a great mathematician and you are tasked with finding if an solution to a given maths question is correct or not. Your response should be 'True' if solution is correct, otherwise 'False'. Below is Question and Solution.
Question:
A line is parameterized by
\[\begin{pmatrix} x \\ y \end{pmatrix} = \begin{pmatrix} 2 \\ 3 \end{pmatrix} + t \begin{pmatrix} -1 \\ 5 \end{pmatrix}.\]A second line is parameterized by
\[\begin{pmatrix} x \\ y \end{pmatrix} = \begin{pmatrix} 0 \\ 7 \end{pmatrix} + u \begin{pmatrix} -1 \\ 4 \end{pmatrix}.\]Find the point where the lines intersect.
Solution:
First, we need to solve the system of equations
\[
\begin{aligned}
2 - t &= s\\
3 + 5t &= 7 + 4s
\end{aligned}
\]
by eliminating s.
We'll use sympy.
<llm-code>
from sympy import symbols, solve

# define the variables
t, s = symbols('t s')

# define the equations
equation1 = 2 - t - s
equation2 = 3 + 5*t - (7 + 4*s)

# solve the equations
solutions = solve([equation1, equation2], (t, s))
p

[' False']

In [ ]:
## Running inference in full Validation set

final_response = []
correct_predictions = 0  # Initialize correct predictions count
for i in range(len(validation_dataset)):
    FastLanguageModel.for_inference(model)
    input_prompt=validation_dataset['text'][i]
    inputs = tokenizer([
          input_prompt
      ], return_tensors = "pt").to("cuda")

    input_shape = inputs['input_ids'].shape
    input_token_len = input_shape[1] # 1 because of batch
    outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
    response = tokenizer.batch_decode([outputs[0][input_token_len:]], skip_special_tokens=True)
    final_response.append(response[0])

Unsloth: Input IDs of shape torch.Size([1, 1146]) with length 1146 > the model's max sequence length of 1024.
We shall truncate it ourselves. It's imperative if you correct this issue first.
Unsloth: Input IDs of shape torch.Size([1, 1094]) with length 1094 > the model's max sequence length of 1024.
We shall truncate it ourselves. It's imperative if you correct this issue first.
Unsloth: Input IDs of shape torch.Size([1, 1050]) with length 1050 > the model's max sequence length of 1024.
We shall truncate it ourselves. It's imperative if you correct this issue first.
Unsloth: Input IDs of shape torch.Size([1, 1060]) with length 1060 > the model's max sequence length of 1024.
We shall truncate it ourselves. It's imperative if you correct this issue first.
Unsloth: Input IDs of shape torch.Size([1, 1140]) with length 1140 > the model's max sequence length of 1024.
We shall truncate it ourselves. It's imperative if you correct this issue first.
Unsloth: Input IDs of shape torch.Size([1, 10

## VALIDATION ACCURACY

In [ ]:
def extract_last_true_false(s):
    matches = re.findall(r'\b(True|False)\b', s, flags=re.IGNORECASE)
    if matches:
        last_match = matches[-1].lower()
        return True if last_match == 'true' else False
    else:
        return False

In [ ]:
import re
validation_prediction_list = [extract_last_true_false(s) for s in final_response]

In [ ]:
validation_truth_list=dataset['validation']['is_correct']

In [ ]:
# Ensure both lists have the same length
assert len(validation_prediction_list) == len(validation_truth_list), "Lists must have the same length."

# Calculate the number of correct predictions
correct_predictions = sum(
    pred == truth for pred, truth in zip(validation_prediction_list, validation_truth_list)
)

# Calculate accuracy
accuracy = correct_predictions / len(validation_truth_list)

print(f"Validation Accuracy: {accuracy:.2%}")


Validation Accuracy: 79.20%


In [ ]:
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset

# Load the official test set
test_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")
predictions = []

# Create the prompt template for inference (no answer included)
inference_prompt = """You are a meticulous math expert. Your goal is to verify the *entire reasoning* of the solution, not just the final answer.
Read the problem, then carefully analyze each step of the solution for logical errors, calculation mistakes, or incorrect reasoning.
Based on your step-by-step analysis, determine if the solution is correct.

Answer with ONLY True or False.

### Problem:
{}

### Solution:
{}

### Output:
"""

def parse_output(response_text):
    output_part = response_text.split("Output:\n")[-1]
    if 'true' in output_part.lower():
        return True
    return False

for example in tqdm(test_dataset):
    question = example["question"]
    solution = example["solution"]

    prompt = inference_prompt.format(question, str(solution))
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    # Generate the prediction
    outputs = model.generate(**inputs, max_new_tokens=8, use_cache=True)
    response_text = tokenizer.batch_decode(outputs)[0]

    # Parse the prediction and add it to our list
    prediction = parse_output(response_text)
    predictions.append(prediction)

# Create the submission DataFrame
submission = pd.DataFrame({
    'ID': range(len(predictions)),
    'is_correct': predictions
})

# Save the DataFrame to a CSV file
submission.to_csv('submission.csv', index=False)

print("\nSubmission file 'submission.csv' created successfully!")
print("You can now download this file and submit it to the Kaggle competition.")

 11%|█         | 1064/10000 [04:59<44:34,  3.34it/s]Unsloth: Input IDs of shape torch.Size([1, 1057]) with length 1057 > the model's max sequence length of 1048.
We shall truncate it ourselves. It's imperative if you correct this issue first.
 13%|█▎        | 1274/10000 [05:59<40:19,  3.61it/s]Unsloth: Input IDs of shape torch.Size([1, 1224]) with length 1224 > the model's max sequence length of 1048.
We shall truncate it ourselves. It's imperative if you correct this issue first.
 16%|█▌        | 1572/10000 [07:22<39:56,  3.52it/s]Unsloth: Input IDs of shape torch.Size([1, 1066]) with length 1066 > the model's max sequence length of 1048.
We shall truncate it ourselves. It's imperative if you correct this issue first.
 23%|██▎       | 2329/10000 [10:56<36:13,  3.53it/s]Unsloth: Input IDs of shape torch.Size([1, 1239]) with length 1239 > the model's max sequence length of 1048.
We shall truncate it ourselves. It's imperative if you correct this issue first.
 30%|███       | 3050/10000 


Submission file 'submission.csv' created successfully!
You can now download this file and submit it to the Kaggle competition.


In [ ]:
save_path = "/content/5hr-submission-model-1"

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print("✅ Model and tokenizer saved to", save_path)

✅ Model and tokenizer saved to /content/5hr-submission-model-1


In [ ]:
import os
import shutil
from google.colab import files

save_path = "/content/5hr-submission-model-1"
# 🗜️ Zip the folder
zip_path = f"{save_path}.zip"
shutil.make_archive(save_path, 'zip', save_path)

# 💾 Download the zip file
files.download(zip_path)
print("⬇️ Download started! Your model zip file is ready.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

⬇️ Download started! Your model zip file is ready.


In [ ]:
save_path = "/content/5hr-submission-model-2"

trainer.model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print("✅ Model and tokenizer saved to", save_path)

✅ Model and tokenizer saved to /content/5hr-submission-model-2


In [ ]:
print(save_path)

/content/5hr-submission-model-1


In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = save_path,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.10.12: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
new_training_set = shuffled_train.select(range(82000, 164000))


In [ ]:
new_training_set = new_training_set.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/82000 [00:00<?, ? examples/s]

In [ ]:
len(new_training_set['text'])

82000

In [ ]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 80,000 | Num Epochs = 1 | Total steps = 10,000
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 83,886,080 of 8,114,147,328 (1.03% trained)


Step,Training Loss
1,0.501500
2,0.430400
3,0.417200
4,0.407600
5,0.332800
6,0.442100
7,0.276000
8,0.437700
9,0.326400
10,0.428800


In [ ]:
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset

# Load the official test set
test_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")
predictions = []

# Create the prompt template for inference (no answer included)
inference_prompt = """You are a meticulous math expert. Your goal is to verify the *entire reasoning* of the solution, not just the final answer.
Read the problem, then carefully analyze each step of the solution for logical errors, calculation mistakes, or incorrect reasoning.
Based on your step-by-step analysis, determine if the solution is correct.

Answer with ONLY True or False.

### Problem:
{}

### Solution:
{}

### Output:
"""

def parse_output(response_text):
    output_part = response_text.split("Output:\n")[-1]
    if 'true' in output_part.lower():
        return True
    return False

for example in tqdm(test_dataset):
    question = example["question"]
    solution = example["solution"]

    prompt = inference_prompt.format(question, str(solution))
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    # Generate the prediction
    outputs = model.generate(**inputs, max_new_tokens=8, use_cache=True)
    response_text = tokenizer.batch_decode(outputs)[0]

    # Parse the prediction and add it to our list
    prediction = parse_output(response_text)
    predictions.append(prediction)

# Create the submission DataFrame
submission = pd.DataFrame({
    'ID': range(len(predictions)),
    'is_correct': predictions
})

# Save the DataFrame to a CSV file
submission.to_csv('submission.csv', index=False)

file_path = '/content/submission.csv'

files.download(file_path)

print("\nSubmission file 'submission.csv' created successfully!")
print("You can now download this file and submit it to the Kaggle competition.")

In [ ]:
save_path = "/content/10hr-submission-model"

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print("✅ Model and tokenizer saved to", save_path)

✅ Model and tokenizer saved to /content/10hr-submission-model


In [ ]:
import os
import shutil
from google.colab import files

# 🗜️ Zip the folder
zip_path = f"{save_path}.zip"
shutil.make_archive(save_path, 'zip', save_path)

# 💾 Download the zip file
files.download(zip_path)
print("⬇️ Download started! Your model zip file is ready.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

⬇️ Download started! Your model zip file is ready.


In [ ]:
import zipfile
import os

zip_path = "/content/10hr-submission-model.zip"   # 👈 your uploaded zip file
extract_path = "/content/10hr-submission-model"   # 👈 where to unzip

# Create folder if not exists
os.makedirs(extract_path, exist_ok=True)

# Unzip
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"✅ Unzipped to: {extract_path}")

✅ Unzipped to: /content/10hr-submission-model


In [ ]:
save_path = "/content/10hr-submission-model"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = save_path,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Prepare the loaded model for faster inference
FastLanguageModel.for_inference(model)

print(f"Model and tokenizer loaded from: {save_path}")

==((====))==  Unsloth 2025.10.12: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

Unsloth 2025.10.12 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Model and tokenizer loaded from: /content/10hr-submission-model


In [ ]:
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset

# Load the official test set
test_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")
predictions = []

# Create the prompt template for inference (no answer included)
inference_prompt = """You are a meticulous math expert. Your goal is to verify the *entire reasoning* of the solution, not just the final answer.
Read the problem, then carefully analyze each step of the solution for logical errors, calculation mistakes, or incorrect reasoning.
Based on your step-by-step analysis, determine if the solution is correct.

Answer with ONLY True or False.

### Problem:
{}

### Solution:
{}

### Output:
"""

def parse_output(response_text):
    output_part = response_text.split("Output:\n")[-1]
    if 'true' in output_part.lower():
        return True
    return False

for example in tqdm(test_dataset):
    question = example["question"]
    solution = example["solution"]

    prompt = inference_prompt.format(question, str(solution))
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    # Generate the prediction
    outputs = model.generate(**inputs, max_new_tokens=8, use_cache=True)
    response_text = tokenizer.batch_decode(outputs)[0]

    # Parse the prediction and add it to our list
    prediction = parse_output(response_text)
    predictions.append(prediction)

# Create the submission DataFrame
submission = pd.DataFrame({
    'ID': range(len(predictions)),
    'is_correct': predictions
})

# Save the DataFrame to a CSV file
submission.to_csv('submission.csv', index=False)

file_path = '/content/submission.csv'

files.download(file_path)

print("\nSubmission file 'submission.csv' created successfully!")
print("You can now download this file and submit it to the Kaggle competition.")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

data/train-00001-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/3.65M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

 11%|█         | 1064/10000 [05:09<42:14,  3.53it/s]Unsloth: Input IDs of shape torch.Size([1, 1057]) with length 1057 > the model's max sequence length of 1048.
We shall truncate it ourselves. It's imperative if you correct this issue first.
 13%|█▎        | 1274/10000 [06:09<41:20,  3.52it/s]Unsloth: Input IDs of shape torch.Size([1, 1224]) with length 1224 > the model's max sequence length of 1048.
We shall truncate it ourselves. It's imperative if you correct this issue first.
 16%|█▌        | 1572/10000 [07:34<39:23,  3.57it/s]Unsloth: Input IDs of shape torch.Size([1, 1066]) with length 1066 > the model's max sequence length of 1048.
We shall truncate it ourselves. It's imperative if you correct this issue first.
 23%|██▎       | 2329/10000 [11:08<35:58,  3.55it/s]Unsloth: Input IDs of shape torch.Size([1, 1239]) with length 1239 > the model's max sequence length of 1048.
We shall truncate it ourselves. It's imperative if you correct this issue first.
 30%|███       | 3050/10000 

NameError: name 'files' is not defined